In [1]:
import pandas as pd
import statsmodels.api as sm 
from statsmodels.formula.api import ols
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [2]:
actor_names = pd.read_csv('data/tsv.gz_files/name.basics.tsv.gz', compression='gzip', sep='\t')
ratings = pd.read_csv('data/tsv.gz_files/title.ratings.tsv.gz', compression='gzip', sep='\t')
title = pd.read_csv('data/tsv.gz_files/title.basics.tsv.gz', compression='gzip', sep='\t')
principles = pd.read_csv('data/tsv.gz_files/title.principals.tsv.gz',compression='gzip', sep='\t')    

C:\Users\vince\AppData\Local\Temp\ipykernel_29768\2967281740.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title = pd.read_csv('data/tsv.gz_files/title.basics.tsv.gz', compression='gzip', sep='\t')


Finding the top rated movie of 2020

In [3]:
principles.head(5)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N


In [4]:
actor_names.describe()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
count,13889075,13889067,13889075,13889075,13889075,13889075
unique,13889075,10671069,544,487,23050,5788503
top,nm9993719,Alex,\N,\N,\N,\N
freq,1,523,13260453,13653652,2690188,1563709


In [5]:
actor_names.columns

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')

In [6]:
actor_names.head(5)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


In [7]:
ratings.head(5)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2095
1,tt0000002,5.6,283
2,tt0000003,6.5,2104
3,tt0000004,5.4,183
4,tt0000005,6.2,2841


In [8]:
title.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11179851 entries, 0 to 11179850
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 767.7+ MB


In [10]:
actor_names.head(5)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"


Finding the top rated movie of 2020

In [11]:
#filtering for movies in 2020
movies_in_2020 = title[(title['startYear']=='2020') & (title['titleType'] =='movie')]
movies_in_2020

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
61105,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,\N,70,Drama
233817,tt0244158,movie,Radha Krishna,Radha Krishna,0,2020,\N,154,Drama
280880,tt0293513,movie,"Pit Bull: A Tale of Lust, Murder and Revenge",Pit Bull,0,2020,\N,\N,\N
420916,tt0438755,movie,About Face: The Story of the Jewish Refugee So...,About Face: The Story of the Jewish Refugee So...,0,2020,\N,94,Documentary
428721,tt0446792,movie,Surviving in L.A.,Surviving in L.A.,0,2020,\N,\N,"Comedy,Drama,Romance"
...,...,...,...,...,...,...,...,...,...
11178224,tt9913288,movie,Letters,Letters,0,2020,\N,\N,Drama
11178645,tt9914192,movie,No Gogó do Paulinho,No Gogó do Paulinho,0,2020,\N,98,Comedy
11179533,tt9916190,movie,Safeguard,Safeguard,0,2020,\N,95,"Action,Adventure,Thriller"
11179572,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,\N,84,Thriller


In [12]:
#joining ratings and movie titles table 
merged_ratings_movie_titles = pd.merge(movies_in_2020,ratings,how = "left", on ='tconst')   
merged_ratings_movie_titles  

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,\N,70,Drama,6.5,193.0
1,tt0244158,movie,Radha Krishna,Radha Krishna,0,2020,\N,154,Drama,NaN,NaN
2,tt0293513,movie,"Pit Bull: A Tale of Lust, Murder and Revenge",Pit Bull,0,2020,\N,\N,\N,NaN,NaN
3,tt0438755,movie,About Face: The Story of the Jewish Refugee So...,About Face: The Story of the Jewish Refugee So...,0,2020,\N,94,Documentary,8.2,46.0
4,tt0446792,movie,Surviving in L.A.,Surviving in L.A.,0,2020,\N,\N,"Comedy,Drama,Romance",7.3,30.0
...,...,...,...,...,...,...,...,...,...,...,...
16458,tt9913288,movie,Letters,Letters,0,2020,\N,\N,Drama,NaN,NaN
16459,tt9914192,movie,No Gogó do Paulinho,No Gogó do Paulinho,0,2020,\N,98,Comedy,5.3,312.0
16460,tt9916190,movie,Safeguard,Safeguard,0,2020,\N,95,"Action,Adventure,Thriller",3.6,261.0
16461,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,\N,84,Thriller,5.8,1490.0


In [13]:
#rescaling num votes to a 1-10 scale to match the ratings column 
merged_ratings_movie_titles['weightedRating'] = merged_ratings_movie_titles['numVotes'] /2953991*10 + merged_ratings_movie_titles['averageRating']   
 

In [14]:
#sorting by number of votes and average rating in descending order to get the top rated film of 2020 
merged_ratings_movie_titles.sort_values(['numVotes','averageRating'], ascending= False)[['primaryTitle','averageRating','numVotes','weightedRating']]


,primaryTitle,averageRating,numVotes,weightedRating
14548,Tenet,7.3,609238.0,9.362423
13242,Soul,8.0,393048.0,9.330566
14685,Wonder Woman 1984,5.4,296386.0,6.403341
15281,A Quiet Place Part II,7.2,291256.0,8.185975
15673,Extraction,6.8,270433.0,7.715483
...,...,...,...,...
16437,Future Vamps,NaN,NaN,NaN
16440,The Line,NaN,NaN,NaN
16448,"Take Pains, Be Perfect",NaN,NaN,NaN
16456,The Saving,NaN,NaN,NaN


In [15]:
merged_ratings_movie_titles.sort_values(['numVotes','averageRating'], ascending= False)[['primaryTitle','averageRating','numVotes']]

,primaryTitle,averageRating,numVotes
14548,Tenet,7.3,609238.0
13242,Soul,8.0,393048.0
14685,Wonder Woman 1984,5.4,296386.0
15281,A Quiet Place Part II,7.2,291256.0
15673,Extraction,6.8,270433.0
...,...,...,...
16437,Future Vamps,NaN,NaN
16440,The Line,NaN,NaN
16448,"Take Pains, Be Perfect",NaN,NaN
16456,The Saving,NaN,NaN


In [16]:
#Finding the top rated movie in 2020 
merged_ratings_movie_titles.sort_values('numVotes',ascending= False)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,weightedRating
14548,tt6723592,movie,Tenet,Tenet,0,2020,\N,150,"Action,Sci-Fi,Thriller",7.3,609238.0,9.362423
13242,tt2948372,movie,Soul,Soul,0,2020,\N,100,"Adventure,Animation,Comedy",8.0,393048.0,9.330566
14685,tt7126948,movie,Wonder Woman 1984,Wonder Woman 1984,0,2020,\N,151,"Action,Adventure,Fantasy",5.4,296386.0,6.403341
15281,tt8332922,movie,A Quiet Place Part II,A Quiet Place Part II,0,2020,\N,97,"Drama,Horror,Sci-Fi",7.2,291256.0,8.185975
15673,tt8936646,movie,Extraction,Extraction,0,2020,\N,116,"Action,Crime,Thriller",6.8,270433.0,7.715483
...,...,...,...,...,...,...,...,...,...,...,...,...
16437,tt9886784,movie,Future Vamps,Future Vamps,0,2020,\N,60,Horror,NaN,NaN,NaN
16440,tt9890326,movie,The Line,The Line,0,2020,\N,\N,"Crime,Thriller",NaN,NaN,NaN
16448,tt9896102,movie,"Take Pains, Be Perfect","Take Pains, Be Perfect",0,2020,\N,\N,Documentary,NaN,NaN,NaN
16456,tt9911046,movie,The Saving,The Saving,0,2020,\N,\N,"Action,Drama",NaN,NaN,NaN


In [17]:
# checking for weightedRatings only
# It is skewed towards movies with low number of votes but highly rated
merged_ratings_movie_titles.sort_values(['weightedRating'], ascending= False)[['primaryTitle','averageRating','numVotes','weightedRating']]

,primaryTitle,averageRating,numVotes,weightedRating
6378,The Dancin' Bulldogs,10.0,15.0,10.000051
7591,The Confessors,10.0,9.0,10.000030
11357,30 Days Sober - A Life Without Booze,10.0,9.0,10.000030
14417,Sistas Conversation,10.0,9.0,10.000030
11030,Blind Surfer with Pete Gustin,10.0,8.0,10.000027
...,...,...,...,...
16437,Future Vamps,NaN,NaN,NaN
16440,The Line,NaN,NaN,NaN
16448,"Take Pains, Be Perfect",NaN,NaN,NaN
16456,The Saving,NaN,NaN,NaN


Finding Top Actor

In [18]:
actor_names

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0050419,tt0072308,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"
...,...,...,...,...,...,...
13889070,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
13889071,nm9993716,Essias Loberg,\N,\N,\N,\N
13889072,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
13889073,nm9993718,Aayush Nair,\N,\N,cinematographer,tt8736744


In [19]:
principles = principles[principles['category'] =='actor']

In [20]:
media_in_2020 = title[(title['startYear']=='2020')]

In [21]:
principles_with_actor_names = pd.merge(principles,actor_names,how = "inner", on ='nconst')

In [22]:
media_with_ratings = pd.merge(media_in_2020,ratings,how = "inner", on ='tconst')

In [23]:
#creating the weights
media_with_ratings['weightedRating'] =  media_with_ratings['numVotes'] /2953991 * 10 + media_with_ratings ['averageRating']

In [24]:
#joining on merged tables
merged_film = pd.merge(principles_with_actor_names,media_with_ratings,how = "inner", on ='tconst')

In [25]:
#checking without proper threshold
merged_film.groupby('primaryName')['weightedRating'].sum().reset_index().sort_values(['weightedRating'], ascending= False)

,primaryName,weightedRating
16505,Dee Bradley Baker,2456.138911
69740,Zeno Robinson,1754.877343
8416,Billy Kametz,1209.684147
64523,Tom Kenny,1081.670550
19607,Eric Bauza,1064.855197
...,...,...
59206,Shahrooz Aghaeipour,1.000034
43879,Mehrdad Absalan,1.000034
7535,Behruz Afkhami,1.000020
7282,Barnabás Réti,1.000017


In [26]:
#create the aggregated table 
aggregated_film = merged_film.groupby('primaryName').agg({'weightedRating':['mean','sum'],'numVotes':'sum','tconst':'count'}).reset_index()


In [27]:
#final aggregation with thresholds
aggregated_film[(aggregated_film[('numVotes', 'sum')] > 500000) 
                 & (aggregated_film[('tconst', 'count')]>10)   
                 ].sort_values(('weightedRating','mean'), ascending=False)

primaryName weightedRating             numVotes tconst
                                        mean         sum      sum  count
51154            Pedro Pascal       8.310380   99.724556   568512     12
63485  Thomas Brodie-Sangster       8.210858   98.530294   629287     12
49033     Okieriete Onaodowan       7.961451  214.959186   549202     27
8297                Bill Camp       7.875065  291.377395   790900     37
62495        Takahiro Sakurai       7.365088  655.492845   500065     89

Genre Trends

In [28]:
media_with_ratings['genres'] = media_with_ratings['genres'].str.split(',')
media_with_ratings = media_with_ratings.explode('genres')
media_with_ratings

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,weightedRating
0,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,\N,70,Drama,6.5,193,6.500653
1,tt0206169,short,The News No. 3,The News No. 3,0,2020,\N,3,Short,7.2,6,7.200020
2,tt0222607,tvSeries,Noticiero 24 horas,Noticiero 24 horas,0,2020,\N,\N,News,7.7,6,7.700020
3,tt0280237,tvSeries,Big Brother,Big Brother,0,2020,\N,\N,Game-Show,2.8,98,2.800332
3,tt0280237,tvSeries,Big Brother,Big Brother,0,2020,\N,\N,Reality-TV,2.8,98,2.800332
...,...,...,...,...,...,...,...,...,...,...,...,...
58579,tt9916190,movie,Safeguard,Safeguard,0,2020,\N,95,Adventure,3.6,261,3.600884
58579,tt9916190,movie,Safeguard,Safeguard,0,2020,\N,95,Thriller,3.6,261,3.600884
58580,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,\N,84,Thriller,5.8,1490,5.805044
58581,tt9916362,movie,Coven,Akelarre,0,2020,\N,92,Drama,6.4,5920,6.420041


In [29]:
genres_popular = media_with_ratings[['genres','weightedRating']]
genres_popular

,genres,weightedRating
0,Drama,6.500653
1,Short,7.200020
2,News,7.700020
3,Game-Show,2.800332
3,Reality-TV,2.800332
...,...,...
58579,Adventure,3.600884
58579,Thriller,3.600884
58580,Thriller,5.805044
58581,Drama,6.420041


One-way ANOVA

In [30]:
model = ols('weightedRating ~ C(genres)', 
            data=genres_popular).fit() 
result = sm.stats.anova_lm(model, type=2) 
result

,df,sum_sq,mean_sq,F,PR(>F)
C(genres),27.0,5446.095262,201.707232,108.96641,0.0
Residual,117292.0,217118.695106,1.851096,NaN,NaN


Topic Modelling

In [31]:
summaries = pd.read_csv('data/imdb_top_1000.csv')

In [32]:
summary_ratings = pd.merge(summaries,media_with_ratings,how = "inner", left_on ='Series_Title', right_on = 'primaryTitle')
summary_ratings = summary_ratings[["weightedRating", "Overview"]]

In [33]:
#the themes of the top rated movies
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(summary_ratings['Overview'])


number_of_themes = 4
nmf = NMF(n_components=number_of_themes, random_state=42)  # Assuming 3 topics/themes
nmf_features = nmf.fit_transform(tfidf)


for index, topic in enumerate(nmf.components_):
    print(f"Theme #{index}:")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-5:]])


summary_ratings['theme'] = nmf_features.argmax(axis=1)

theme_ratings = summary_ratings.groupby('theme')['weightedRating'].mean().reset_index()


top_themes = theme_ratings.sort_values(by='weightedRating', ascending=False)
print(top_themes)

Theme #0:
['1920s', 'vivid', 'arm', 'angeles', 'los']
Theme #1:
['experiencing', 'strangers', 'sexually', 'hurting', 'men']
Theme #2:
['doing', 'way', 'insane', 'murderously', 'invisible']
Theme #3:
['land', 'man', 'young', 'town', 'magical']
   theme  weightedRating
0      0        7.728921
2      2        7.674547
3      3        7.289298
1      1        7.050412
